### Monitoring and Observability

In [1]:
from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import SERVICE_NAME, Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor

otel_exporter = OTLPSpanExporter(endpoint="localhost:4317", insecure=True)
tracer_provider = TracerProvider(
    resource=Resource.create({SERVICE_NAME: "my-service"})
)
span_processor = BatchSpanProcessor(otel_exporter)
tracer_provider.add_span_processor(span_processor)
trace.set_tracer_provider(tracer_provider)

In [2]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.messages import AgentEvent, ChatMessage, TextMessage
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.base import TaskResult
from autogen_core import CancellationToken

import os

from dotenv import load_dotenv

In [3]:
load_dotenv(override=True)

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

if not all([azure_openai_api_key, azure_openai_endpoint, azure_openai_deployment_name, azure_openai_api_version]):
    raise ValueError("""
        Please set all required environment variables: 
            AZURE_OPENAI_API_KEY, 
            AZURE_OPENAI_ENDPOINT, 
            AZURE_OPENAI_DEPLOYMENT_NAME, 
            AZURE_OPENAI_API_VERSION
        """)


In [4]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment_name,
    model=azure_openai_deployment_name,
    api_key=azure_openai_api_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=azure_openai_api_version,
)

In [5]:
assistant_agent = AssistantAgent(
    name="Assistant",
    model_client=model_client,
    system_message="You are a fun fact expert about animals. ",
)

tracer = trace.get_tracer("my-service")

with tracer.start_as_current_span("runtime"):
    response = await assistant_agent.on_messages(
        [
            TextMessage(
                content="Tell me a fun fact about Dolphins.",
                source="user",
            )
        ],
        cancellation_token=CancellationToken(),
    )

    print(response.chat_message.content)

Dolphins are known for their incredible intelligence and social behaviors! One fun fact is that dolphins have been observed using tools—specifically, some species use marine sponges to protect their snouts while foraging on the seafloor. This behavior not only showcases their problem-solving abilities but also indicates a cultural aspect, as young dolphins learn this technique from their mothers and other individuals in their pods!
